In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pprint
%matplotlib inline

In [107]:
imgs = []

try:
    for i in range(1, 13):
        filename = "test" + str(i)
        #filename = "test5"
        img = cv2.imread("./images/"+filename+".jpg", cv2.IMREAD_COLOR)

        #print(img.shape)

        # image resize 360 x 219
        #img = cv2.resize(img, (500, 500), interpolation=cv2.INTER_LINEAR)

        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #cv2.imshow("img_gray", img_gray)
        #img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
        #cv2.imshow("img_blur", img_blur)

        ## 적당한 threshold 값을 찾아서
        #ret, img_th = cv2.threshold(img_blur, 150, 230, cv2.THRESH_BINARY_INV)
        #th2 = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,15,2)
        #ret1, img_th1 = cv2.threshold(img_gray, 150, 230, cv2.THRESH_BINARY_INV)
        #ret2, img_th2 = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        #ret3, img_th3 = cv2.threshold(img_blur, 150, 230, cv2.THRESH_BINARY_INV)
        ret4, img_th4 = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        #img_th4 = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 5, 10)

        #img_lapl = cv2.Laplacian(img_th4, cv2.CV_32F)

        kernel = np.ones((3, 3), np.uint8)
        img_morph = cv2.morphologyEx(img_th4, cv2.MORPH_CLOSE, kernel)
        img_lapl = cv2.Laplacian(img_morph, cv2.CV_32F)

        cv2.imshow("image"+str(i), img_morph)
        #cv2.imshow("[morph]image"+str(i), img_morph)

        # 대상은 흰색, 배경은 까만색이어야 컨투어 찾기 가능
        contours, hierachy = cv2.findContours(img_morph.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        img_cont = cv2.drawContours(img.copy(), contours, -1, (0,255,0), 1)
        
        cv2.imshow("[cont]image"+str(i), img_cont)
        
        ###################### boundingRect test [start] ######################
        #x, y, w, h = cv2.boundingRect(contours[0])
        #print(x, y, w, h)
        #testimg = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        #cv2.imshow("img"+str(i), testimg)
        ###################### boundingRect test [end] ########################
        
        # 사각형 그리기
        rects = [cv2.boundingRect(each) for each in contours]

        tmp = [w*h for (x,y,w,h) in rects]
        tmp.sort()

        # 제일 큰 아이 두개를 빼고, 나머지의 넓이를 찾을 수 있습니다. 그래서 넓이값으로 다시 한 번 더 걸러주는거죠...
        #rects = [(x,y,w,h) for (x,y,w,h) in rects if ((w*h>15000)and(w*h<500000))]
        rects = [(x,y,w,h) for (x,y,w,h) in rects]
#        
        print(rects)
        print()
#        
        img_result = []
        img_for_class = img.copy()

        margin_pixel = 30

        for rect in rects:
            #[y:y+h, x:x+w]
            img_result.append(
                img_for_class[rect[1]-margin_pixel : rect[1]+rect[3]+margin_pixel, 
                              rect[0]-margin_pixel : rect[0]+rect[2]+margin_pixel])

            # Draw the rectangles
            cv2.rectangle(img, (rect[0], rect[1]), 
                          (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 2)
#
        imgs.append(img)
        
except:
    print("Exception!!!!!!!!!")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
cv2.waitKey(0)
cv2.destroyAllWindows()

[(1114, 527, 8, 9), (522, 527, 9, 16), (243, 527, 8, 9), (1060, 518, 42, 22), (755, 518, 42, 21), (623, 518, 41, 21), (415, 518, 41, 21), (1054, 508, 55, 7), (333, 486, 32, 43), (672, 484, 55, 49), (274, 484, 31, 44), (922, 483, 36, 33), (746, 483, 36, 33), (406, 483, 36, 34), (863, 482, 55, 57), (1060, 481, 43, 24), (996, 480, 55, 60), (805, 480, 49, 61), (466, 480, 54, 60), (374, 480, 8, 61), (312, 480, 17, 61), (185, 480, 54, 61), (931, 479, 41, 62), (787, 479, 8, 46), (616, 479, 47, 46), (554, 479, 56, 60), (447, 479, 8, 46), (127, 479, 46, 62), (65, 479, 56, 61), (431, 342, 9, 9), (813, 330, 17, 8), (359, 330, 17, 8), (765, 314, 33, 51), (682, 314, 48, 37), (597, 314, 9, 37), (732, 313, 31, 39), (647, 313, 32, 52), (612, 313, 31, 38), (541, 313, 49, 38), (506, 313, 30, 39), (463, 302, 37, 49), (396, 302, 31, 49), (597, 299, 9, 9), (352, 160, 9, 10), (1067, 158, 9, 9), (713, 157, 11, 18), (352, 139, 9, 9), (921, 130, 48, 37), (819, 130, 32, 37), (492, 130, 33, 51), (1031, 129, 30, 

[(58, 351, 27, 4), (88, 331, 15, 21), (59, 331, 25, 15), (185, 328, 27, 31), (126, 328, 56, 31), (107, 328, 8, 31), (29, 328, 28, 31), (57, 302, 5, 7), (260, 294, 27, 14), (328, 281, 27, 24), (231, 281, 27, 24), (30, 281, 24, 27), (416, 280, 27, 28), (263, 280, 21, 11), (192, 280, 27, 25), (133, 280, 27, 29), (104, 280, 27, 28), (66, 280, 27, 25), (387, 278, 26, 31), (357, 278, 25, 30), (289, 278, 26, 31), (163, 278, 28, 31), (49, 278, 8, 23), (187, 142, 8, 3), (287, 130, 24, 25), (254, 130, 24, 22), (230, 130, 21, 24), (131, 130, 21, 25), (78, 130, 25, 25), (313, 129, 50, 26), (247, 129, 4, 19), (204, 129, 22, 26), (155, 129, 25, 26), (105, 129, 22, 26), (183, 107, 8, 4), (251, 96, 22, 24), (226, 96, 21, 24), (310, 95, 49, 25), (283, 95, 24, 25), (268, 95, 7, 19), (244, 95, 3, 19), (200, 95, 22, 25), (145, 95, 22, 26), (120, 95, 21, 25), (94, 95, 21, 25), (181, 73, 8, 4), (127, 62, 32, 23), (108, 62, 16, 22), (249, 61, 21, 25), (224, 61, 21, 25), (282, 60, 75, 27), (266, 60, 7, 19), (

In [108]:
# 이미지 한번에 출력
for i in range(0, len(imgs)):
    cv2.imshow("image"+str(i), imgs[i])
    
cv2.waitKey(0)
cv2.destroyAllWindows()

In [214]:
tmp = [w*h for (x,y,w,h) in rects]
tmp.sort()
tmp

[374, 1254, 1474, 2646, 3165, 3420, 3500, 4150, 4700, 7303, 14348, 14348]

In [215]:
# 제일 큰 아이 두개를 빼고, 나머지의 넓이를 찾을 수 있습니다. 그래서 넓이값으로 다시 한 번 더 걸러주는거죠...
#rects = [(x,y,w,h) for (x,y,w,h) in rects if ((w*h>15000)and(w*h<500000))]
rects = [(x,y,w,h) for (x,y,w,h) in rects]
rects

[(345, 292, 66, 19),
 (96, 261, 45, 76),
 (270, 239, 67, 109),
 (85, 223, 67, 22),
 (163, 174, 35, 100),
 (354, 167, 47, 100),
 (22, 162, 22, 17),
 (91, 147, 49, 54),
 (279, 146, 50, 83),
 (420, 138, 68, 211),
 (201, 138, 15, 211),
 (11, 138, 68, 211)]

In [216]:
img_result = []
img_for_class = img.copy()

margin_pixel = 60
print(rects)
for rect in rects:
    #[y:y+h, x:x+w]
    img_result.append(
        img_for_class[rect[1]-margin_pixel : rect[1]+rect[3]+margin_pixel, 
                      rect[0]-margin_pixel : rect[0]+rect[2]+margin_pixel])
    
    # Draw the rectangles
    cv2.rectangle(img, (rect[0], rect[1]), 
                  (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 5)

#plt.figure(figsize=(15,12))
#plt.imshow(img)
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[(345, 292, 66, 19), (96, 261, 45, 76), (270, 239, 67, 109), (85, 223, 67, 22), (163, 174, 35, 100), (354, 167, 47, 100), (22, 162, 22, 17), (91, 147, 49, 54), (279, 146, 50, 83), (420, 138, 68, 211), (201, 138, 15, 211), (11, 138, 68, 211)]
